In [34]:
%load_ext autoreload
%autoreload 2

import torch

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
import main
main.run('misc/experiments/ssdnet/params.json')

Created SSDNet model succesfully!
Number of epochs: 10
Total number of parameters of model:  3052988
Model loaded successfully
loading annotations into memory...
Done (t=2.58s)
creating index...
index created!


AttributeError: 'list' object has no attribute 'squeeze'

In [3]:
import main
main.run('misc/experiments/ssdnet/params.json')

Created SSDNet model succesfully!
Number of epochs: 10
Total number of parameters of model:  4428176


In [ ]:
from architectures.backbones import MobileNet

In [ ]:
model = MobileNet.mobilenet_v2(pretrained=True)

In [ ]:
a = torch.rand(1,3,512,512)

# default os=32, expect 512//32 with 1280 channels output
b = model(a)
print(b.shape)

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print(total_params)

In [ ]:
model_half_width = MobileNet.mobilenet_v2(width_mult=0.5)
total_params = sum(p.numel() for p in model_half_width.parameters())
print(total_params)

In [ ]:
from train.helpers import *

anchors, grid_sz = create_anchors()
anchor_cnr = hw2corners(anchors[:,:2], anchors[:,2:])
print(anchor_cnr, anchor_cnr.shape)

In [ ]:
overlaps = jaccard(anchor_cnr[:3,:], anchor_cnr)

In [ ]:
print(overlaps)

In [ ]:
print(overlaps.max(0))
print(overlaps.max(1))

In [ ]:
import torch
from data.dataloaders import get_dataloaders
valid_dataloader = get_dataloaders()
x, y = next(iter(valid_dataloader))

def prepare_gt(y):
    gt_bbox, gt_clas = [], []
    for obj in y:
        gt_bbox.append(obj['bbox'])
        gt_clas.append(obj['category_id'])
    return [torch.FloatTensor(gt_bbox), torch.IntTensor(gt_clas)]
gt = prepare_gt(y)
overlaps = jaccard(gt[0]/400, anchor_cnr)

In [ ]:
print(gt[0]/400, gt[0].shape) 
print(anchor_cnr, anchor_cnr.shape)

In [ ]:
print(overlaps, overlaps.shape)

In [ ]:
gt[0] = gt[0][:10,:]
gt[1] = gt[1][:10]

In [ ]:
pred = [torch.rand(16,3), torch.rand(16,4)]

In [ ]:
from train.loss_fn import *
ssd_loss(pred,gt)

In [ ]:
a = torch.rand(16,4) * 200
b = gt[0]
print(a, a.shape)
print(b, b.shape)

In [ ]:
overlap = jaccard(anchor_cnr,anchor_cnr+0.05)

In [ ]:
print(overlap, overlap.shape)

In [ ]:
print(anchor_cnr,anchor_cnr+0.05)

In [8]:
import main
main.run('misc/experiments/ssdnet/params.json')

Created SSDNet model succesfully!
loading annotations into memory...
Done (t=1.36s)
creating index...
index created!
Train start...


AttributeError: 'dict' object has no attribute 'to'

In [ ]:
from architectures.models import SSDNet
from data import dataloaders
model = SSDNet.SSD_Head()
train_loader, valid_loader = dataloaders.get_dataloaders()

In [ ]:
from train.helpers import *
x, y = next(iter(valid_loader))

In [ ]:
gt = y

In [ ]:
gt[0], gt[0].shape

In [ ]:
x.shape, anchor_cnr.shape

In [ ]:
overlaps = jaccard(gt[0].squeeze(), anchor_cnr)

In [ ]:
overlaps, overlaps.shape

In [ ]:
for line in overlaps:
    print(torch.max(line))

In [ ]:
overlaps.max(0), overlaps.max(0)[0].shape

In [ ]:
actn_to_bb(torch.rand(16,4), anchors, 0.25)

In [ ]:
import torchvision
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from data.dataset import CocoDetection

composed_transforms = transforms.Compose([transforms.ToTensor()])

validation_dataset = CocoDetection(root='../../COCO/val2017/val2017',
                                          annFile='../../COCO/annotations_trainval2017/annotations/instances_val2017.json',
                                          transform=composed_transforms)

In [ ]:
x,y = next(iter(validation_dataset))
x.size
len(y)

In [ ]:
y[0]

In [ ]:
y[1]

In [ ]:
x.shape

In [ ]:
import cv2

cv2.rectangle(img, (x1, y1), (x2, y2), (255,0,0), 2)

In [ ]:
from train.helpers import *
from data.dataloaders import get_dataloaders

    
valid, valid = get_dataloaders()

In [ ]:
%matplotlib inline
visualize_data(valid)

In [9]:
import numpy as np

a = np.array([[1,2,3], [4,5]])
print(a.shape)
print(a[:, :2])
print(a[:, 2:])

(2,)


IndexError: too many indices for array

In [46]:
import torch

bbox = torch.IntTensor([[0,0],[1,0],[2,3],[3,3],[4,1],[5,0],[6,3]])
gt_idx = torch.LongTensor([0,0,2,3,4,1,6,5,0])
gt_overlap = torch.FloatTensor([0,0,0.2,0.7,0.5,0.3,0.1,0.9,0.5])

In [47]:
pos = gt_overlap > 0.4
print(pos)

tensor([False, False, False,  True,  True, False, False,  True,  True])


In [48]:
pos_idx = torch.nonzero(pos)[:, 0]
print(pos_idx)

tensor([3, 4, 7, 8])


In [49]:
raw_bb = bbox[gt_idx]
print(raw_bb)

tensor([[0, 0],
        [0, 0],
        [2, 3],
        [3, 3],
        [4, 1],
        [1, 0],
        [6, 3],
        [5, 0],
        [0, 0]], dtype=torch.int32)


In [50]:
final_bb = raw_bb[pos_idx]
print(final_bb)

tensor([[3, 3],
        [4, 1],
        [5, 0],
        [0, 0]], dtype=torch.int32)


In [51]:
final2_bb = bbox[pos_idx]
print(final2_bb)

IndexError: index 7 is out of bounds for dimension 0 with size 7

In [52]:
input = torch.randn(3, requires_grad=True)
print(input)
target = torch.empty(3).random_(2)
print(target)

tensor([ 0.7568,  1.9503, -0.2873], requires_grad=True)
tensor([0., 0., 1.])


In [87]:
a = np.array([1,2])
b = torch.from_numpy(a)

In [88]:
b

tensor([1, 2], dtype=torch.int32)

In [91]:
print(b.grad)

None


In [92]:
print(b.requires_grad)

False


In [95]:
def create_anchors():
    ''' anchors and sizes, 4x4 basic atm '''

    anc_grids = [4,2,1]
    anc_zooms = [0.75, 1., 1.3]
    anc_ratios = [(1.,1.), (1.,0.5), (0.5,1.)]
    anchor_scales = [(anz*i,anz*j) for anz in anc_zooms for (i,j) in anc_ratios]
    anc_offsets = [1/(o*2) for o in anc_grids]
    k = len(anchor_scales)

    anc_x = np.concatenate([np.repeat(np.linspace(ao, 1-ao, ag), ag)
                        for ao,ag in zip(anc_offsets,anc_grids)])
    anc_y = np.concatenate([np.tile(np.linspace(ao, 1-ao, ag), ag)
                        for ao,ag in zip(anc_offsets,anc_grids)])
    anc_ctrs = np.repeat(np.stack([anc_x,anc_y], axis=1), k, axis=0)

    anc_sizes  =   np.concatenate([np.array([[o/ag,p/ag] for i in range(ag*ag) for o,p in anchor_scales])
               for ag in anc_grids])
    grid_sizes = torch.from_numpy(np.concatenate([np.array([ 1/ag       for i in range(ag*ag) for o,p in anchor_scales])
                   for ag in anc_grids])).unsqueeze(1)
    anchors = torch.from_numpy(np.concatenate([anc_ctrs, anc_sizes], axis=1)).float()

    return anchors, grid_sizes

In [96]:
a, g = create_anchors()

In [98]:
a.shape

torch.Size([189, 4])

In [99]:
g.shape

torch.Size([189, 1])

In [100]:
g


tensor([[0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2500],
        [0

In [111]:
import torch.nn as nn
import torch.nn.functional as F

def flatten_conv(x,k):
    bs,nf,gx,gy = x.size()
    x = x.permute(0,2,3,1).contiguous()
    return x.view(bs,-1,nf//k)

class OutConv(nn.Module):
    def __init__(self, k, nin, bias):
        super().__init__()
        self.k = k
        self.oconv1 = nn.Conv2d(nin, 3*k, 3, padding=1)
        self.oconv2 = nn.Conv2d(nin, 4*k, 3, padding=1)
        self.oconv1.bias.data.zero_().add_(bias)
        
    def forward(self, x):
        return [flatten_conv(self.oconv1(x), self.k),
                flatten_conv(self.oconv2(x), self.k)]

class StdConv(nn.Module):
    def __init__(self, nin, nout, stride=2, drop=0.1):
        super().__init__()
        self.conv = nn.Conv2d(nin, nout, 3, stride=stride, padding=1)
        self.bn = nn.BatchNorm2d(nout)
        self.drop = nn.Dropout(drop)
        
    def forward(self, x): return self.drop(self.bn(F.relu(self.conv(x))))

class SSD_MultiHead(nn.Module):
    def __init__(self, k, bias, drop=0.1):
        super().__init__()
        self.drop = nn.Dropout(drop)
        self.sconv1 = StdConv(512,256, drop=drop)
        self.sconv2 = StdConv(256,256, drop=drop)
        self.sconv3 = StdConv(256,256, drop=drop)
        self.out0 = OutConv(k, 256, bias)
        self.out1 = OutConv(k, 256, bias)
        self.out2 = OutConv(k, 256, bias)
        self.out3 = OutConv(k, 256, bias)

    def forward(self, x):
        x = self.drop(F.relu(x))
        x = self.sconv1(x)
        o1c,o1l = self.out1(x)
        x = self.sconv2(x)
        o2c,o2l = self.out2(x)
        x = self.sconv3(x)
        o3c,o3l = self.out3(x)
        return [torch.cat([o1c,o2c,o3c], dim=1),
                torch.cat([o1l,o2l,o3l], dim=1)]

In [112]:
a = torch.rand([2,512,7,7])
net = SSD_MultiHead(9,-4.)

In [114]:
b = net(a)

In [116]:
len(b)

2

In [117]:
b[0].shape

torch.Size([2, 189, 3])

In [118]:
b[1].shape

torch.Size([2, 189, 4])

In [125]:
a = [torch.rand(2,2), torch.rand(2,2)]
b = torch.stack(a)

In [126]:
b

tensor([[[0.8194, 0.8068],
         [0.0303, 0.5417]],

        [[0.7930, 0.9366],
         [0.2702, 0.8981]]])

In [127]:
b.shape

torch.Size([2, 2, 2])

In [140]:
b[0]

tensor([[0.8194, 0.8068],
        [0.0303, 0.5417]])

In [31]:
a = torch.rand(1, requires_grad=True)
b = torch.rand(1)

In [32]:
a + b

tensor([0.7342], grad_fn=<AddBackward0>)

In [33]:
c = a + b

In [35]:
print(c.grad)

None


In [36]:
c

tensor([0.7342], grad_fn=<AddBackward0>)

In [43]:
for i in range(1000):
    a = torch.rand((189,4), requires_grad=True)
    b = torch.rand(189,4)
    c = a[:, :2] + b[:, :2] 
    print(torch.max(c))

tensor(1.9458, grad_fn=<MaxBackward1>)
tensor(1.9163, grad_fn=<MaxBackward1>)
tensor(1.9527, grad_fn=<MaxBackward1>)
tensor(1.9322, grad_fn=<MaxBackward1>)
tensor(1.9555, grad_fn=<MaxBackward1>)
tensor(1.9874, grad_fn=<MaxBackward1>)
tensor(1.9315, grad_fn=<MaxBackward1>)
tensor(1.9025, grad_fn=<MaxBackward1>)
tensor(1.9095, grad_fn=<MaxBackward1>)
tensor(1.9032, grad_fn=<MaxBackward1>)
tensor(1.9705, grad_fn=<MaxBackward1>)
tensor(1.9565, grad_fn=<MaxBackward1>)
tensor(1.8790, grad_fn=<MaxBackward1>)
tensor(1.9253, grad_fn=<MaxBackward1>)
tensor(1.9452, grad_fn=<MaxBackward1>)
tensor(1.8860, grad_fn=<MaxBackward1>)
tensor(1.9804, grad_fn=<MaxBackward1>)
tensor(1.8552, grad_fn=<MaxBackward1>)
tensor(1.9415, grad_fn=<MaxBackward1>)
tensor(1.9652, grad_fn=<MaxBackward1>)
tensor(1.9511, grad_fn=<MaxBackward1>)
tensor(1.9114, grad_fn=<MaxBackward1>)
tensor(1.9389, grad_fn=<MaxBackward1>)
tensor(1.9210, grad_fn=<MaxBackward1>)
tensor(1.9149, grad_fn=<MaxBackward1>)
tensor(1.8511, grad_fn=<M

tensor(1.9687, grad_fn=<MaxBackward1>)
tensor(1.9732, grad_fn=<MaxBackward1>)
tensor(1.9341, grad_fn=<MaxBackward1>)
tensor(1.9204, grad_fn=<MaxBackward1>)
tensor(1.8943, grad_fn=<MaxBackward1>)
tensor(1.9667, grad_fn=<MaxBackward1>)
tensor(1.9247, grad_fn=<MaxBackward1>)
tensor(1.9258, grad_fn=<MaxBackward1>)
tensor(1.9698, grad_fn=<MaxBackward1>)
tensor(1.9385, grad_fn=<MaxBackward1>)
tensor(1.9464, grad_fn=<MaxBackward1>)
tensor(1.9421, grad_fn=<MaxBackward1>)
tensor(1.9536, grad_fn=<MaxBackward1>)
tensor(1.8772, grad_fn=<MaxBackward1>)
tensor(1.9234, grad_fn=<MaxBackward1>)
tensor(1.9682, grad_fn=<MaxBackward1>)
tensor(1.9766, grad_fn=<MaxBackward1>)
tensor(1.9428, grad_fn=<MaxBackward1>)
tensor(1.9641, grad_fn=<MaxBackward1>)
tensor(1.9435, grad_fn=<MaxBackward1>)
tensor(1.9247, grad_fn=<MaxBackward1>)
tensor(1.9670, grad_fn=<MaxBackward1>)
tensor(1.8396, grad_fn=<MaxBackward1>)
tensor(1.9215, grad_fn=<MaxBackward1>)
tensor(1.8820, grad_fn=<MaxBackward1>)
tensor(1.9290, grad_fn=<M

tensor(1.8889, grad_fn=<MaxBackward1>)
tensor(1.9600, grad_fn=<MaxBackward1>)
tensor(1.9347, grad_fn=<MaxBackward1>)
tensor(1.9554, grad_fn=<MaxBackward1>)
tensor(1.9253, grad_fn=<MaxBackward1>)
tensor(1.9191, grad_fn=<MaxBackward1>)
tensor(1.9384, grad_fn=<MaxBackward1>)
tensor(1.9236, grad_fn=<MaxBackward1>)
tensor(1.9366, grad_fn=<MaxBackward1>)
tensor(1.9326, grad_fn=<MaxBackward1>)
tensor(1.9166, grad_fn=<MaxBackward1>)
tensor(1.9175, grad_fn=<MaxBackward1>)
tensor(1.9231, grad_fn=<MaxBackward1>)
tensor(1.8868, grad_fn=<MaxBackward1>)
tensor(1.8981, grad_fn=<MaxBackward1>)
tensor(1.9366, grad_fn=<MaxBackward1>)
tensor(1.9697, grad_fn=<MaxBackward1>)
tensor(1.9191, grad_fn=<MaxBackward1>)
tensor(1.9389, grad_fn=<MaxBackward1>)
tensor(1.9394, grad_fn=<MaxBackward1>)
tensor(1.9186, grad_fn=<MaxBackward1>)
tensor(1.9353, grad_fn=<MaxBackward1>)
tensor(1.8424, grad_fn=<MaxBackward1>)
tensor(1.9880, grad_fn=<MaxBackward1>)
tensor(1.9930, grad_fn=<MaxBackward1>)
tensor(1.9369, grad_fn=<M

In [40]:
c.requires_grad

True

In [41]:
torch.max(c)

tensor(1.8850, grad_fn=<MaxBackward1>)

In [48]:
print(c.dtype)

torch.float32


In [52]:
c.to(torch.FloatTensor)

TypeError: to() received an invalid combination of arguments - got (torch.tensortype), but expected one of:
 * (torch.device device, torch.dtype dtype, bool non_blocking, bool copy)
 * (torch.dtype dtype, bool non_blocking, bool copy)
 * (Tensor tensor, bool non_blocking, bool copy)


In [54]:
c.dtype = torch.float64

AttributeError: attribute 'dtype' of 'torch._C._TensorBase' objects is not writable

In [55]:
c = c.type(torch.LongTensor)

In [56]:
c

tensor([[1, 1],
        [0, 0],
        [1, 0],
        [1, 0],
        [1, 1],
        [0, 1],
        [1, 0],
        [1, 1],
        [0, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [0, 1],
        [0, 0],
        [1, 1],
        [0, 0],
        [1, 1],
        [0, 1],
        [1, 1],
        [0, 1],
        [1, 1],
        [1, 0],
        [1, 0],
        [0, 1],
        [0, 1],
        [1, 0],
        [0, 1],
        [1, 1],
        [1, 0],
        [0, 0],
        [0, 1],
        [0, 1],
        [1, 0],
        [0, 1],
        [1, 1],
        [0, 0],
        [0, 1],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [0, 0],
        [1, 1],
        [1, 0],
        [1, 1],
        [0, 1],
        [1, 1],
        [0, 0],
        [0, 1],
        [1, 1],
        [1, 1],
        [0, 1],
        [0, 0],
        [1, 1],
        [1, 0],
        [0, 0],
        [0, 0],
        [1, 0],
        

In [13]:
a = torch.rand(1000)
b = torch.rand(1000, dtype = torch.float64)
print(a.dtype, b.dtype)

torch.float32 torch.float64


In [11]:
b = b.type(torch.DoubleTensor)

In [12]:
print(torch.max(a+b))

tensor(1.9745, dtype=torch.float64)


In [25]:
for i in range(100):
    a = torch.rand((189,4))
    b = torch.rand((189,4), dtype=torch.float64)
    print(torch.max(a[:,:2]+b[:,:2]))

tensor(2.8119e+275, dtype=torch.float64)
tensor(nan, dtype=torch.float64)
tensor(nan, dtype=torch.float64)
tensor(2.8119e+275, dtype=torch.float64)
tensor(9.2142e+269, dtype=torch.float64)
tensor(9.2142e+269, dtype=torch.float64)
tensor(nan, dtype=torch.float64)
tensor(nan, dtype=torch.float64)
tensor(2.2446e+271, dtype=torch.float64)
tensor(2.8119e+275, dtype=torch.float64)
tensor(2.8119e+275, dtype=torch.float64)
tensor(nan, dtype=torch.float64)
tensor(2.8119e+275, dtype=torch.float64)
tensor(9.2142e+269, dtype=torch.float64)
tensor(nan, dtype=torch.float64)
tensor(2.8119e+275, dtype=torch.float64)
tensor(9.2142e+269, dtype=torch.float64)
tensor(2.8119e+275, dtype=torch.float64)
tensor(9.2142e+269, dtype=torch.float64)
tensor(6.8157e+271, dtype=torch.float64)
tensor(nan, dtype=torch.float64)
tensor(9.2142e+269, dtype=torch.float64)
tensor(2.8119e+275, dtype=torch.float64)
tensor(9.2142e+269, dtype=torch.float64)
tensor(6.8157e+271, dtype=torch.float64)
tensor(nan, dtype=torch.float64

In [26]:
print(torch.__version__)

1.3.0


In [29]:
import numpy as np
a = np.array([1.2,2])
b = torch.from_numpy(a).float()
print(b.dtype)

torch.float32


In [9]:
import torch
from architectures.backbones.MobileNet import ConvBNReLU, InvertedResidual, mobilenet_v2
backbone = mobilenet_v2(width_mult=0.2)

In [10]:
a = torch.rand(2,3,320,320)

In [11]:
inter, out = backbone(a)

In [12]:
inter.shape

torch.Size([2, 288, 20, 20])

In [13]:
out.shape

torch.Size([2, 1280, 10, 10])